# Simple ORT UI

This is an attempt at providing an online interface to [OSS Review Toolkit](https://github.com/heremaps/oss-review-toolkit) (ORT), a [Linux Foundation](https://automatingsbom.org/community/projects/) project. It is meant only as a proof of concept and uses basic [Jupyter](https://jupyter.org) [widgets](https://github.com/jupyter-widgets/ipywidgets). It implements all four phases (analyze, scan, evaluate and report) for some sample project, "mime-types", but you should be able to run it on some other, too. (Attention, this is work in progress!) 

In [ ]:
%matplotlib inline

In [1]:
import re
import os
import json
import shutil
import tempfile
import itertools
import subprocess
import textwrap
from os.path import expanduser, exists, join
from functools import partial

In [ ]:
import requests
import pandas as pd
from IPython.display import display, HTML as HTML2
from ipywidgets import (HTML, Tab, HBox, VBox, Button, ButtonStyle,
    Layout, Textarea, Text, Output, Dropdown, Checkbox)
from halo import HaloNotebook
from matplotlib import pyplot

In [ ]:
from ortung import clone_repo, parse_repo_url, HAVE_ORT

In [ ]:
output = Output()
status_spinner = HaloNotebook(spinner='dots')

In [ ]:
@output.capture()
def find_ort(status_spinner, verbose=False):
    """Find local installation of ORT.
    """
    ort_url = "https://github.com/heremaps/oss-review-toolkit"
    s = status_spinner
    s.start("Searching installed ORT...")
    if not HAVE_ORT:
        s.fail()
        s.start(f"ORT not found, please visit {ort_url}.")
    else:
        s.succeed("Found ORT")
    return HAVE_ORT    

In [ ]:
@output.capture()
def clone_repo_clicked(button,
               source=None, url=None, branch=None, dest_dir=None,
               overwrite=True, verbose=False):
    """Clone a repo into some destination folder.
    """
    dest_dir = expanduser(dest_dir or ".")
    if not os.path.exists(dest_dir):
        with output:
            print(f"{dest_dir} does not exist.")
        return

    source = source.value
    url = url.value
    branch = branch.value

    # determine repo full URL, owner and name 
    if re.match("http[s]\://.*\.git", url):
        full_url = url
        owner, repo = full_url[:-4].split("/")[-2:]
    elif source == "GitHub":
        full_url = f"https://github.com/"
        if "/" in url:
            if url.endswith(".git"):
                full_url += url
            else:
                full_url += f"{url}.git"
        owner, repo = full_url[:-4].split("/")[-2:]

    if False: # verbose:
        with output:
            print(f"{url} {dest_dir} {full_url} {owner} {repo}")    
    # with HaloNotebook(text=f"Cloning {full_url}", spinner='dots'):
    #     out = clone_repo(full_url, branch=branch, dest_dir=dest_dir)
    status_spinner.start(f"Cloning {full_url} ...")
    code = requests.head(full_url, allow_redirects=True).status_code
    if code != 200:
        status_spinner.fail()
        status_spinner.start(f"Not found: {url}")
        status_spinner.warn()
    else:
        clone_repo(full_url, branch=branch, dest_dir=dest_dir)
        status_spinner.succeed(f"Cloned {full_url}")

In [ ]:
@output.capture()
def plot_licenses(url=None):
    """Make simple plot.
    """
    url = url.value
    name = parse_repo_url(url)["name"]
    path = f"../tmp/{name}-ort/analyzer/analyzer-result.json"
    if exists(path):
        j = json.load(open(path))
        pkgs = [p["package"] for p in j["analyzer"]["result"]["packages"]]
        df = pd.DataFrame(pkgs)
        licenses = list(itertools.chain.from_iterable(df.declared_licenses))
        df1 = pd.DataFrame(licenses, columns=["License"])
        with output:
            df1["License"].value_counts().plot(kind='barh', title=f"Licenses for {name}")
            pyplot.show()

In [ ]:
@output.capture()
def analyze_clicked(button,
                    url=None,
                    source_dir=None,
                    verbose=False):
    """Analyze a repo.
    """
    source_dir = expanduser(source_dir or ".")
    if not os.path.exists(source_dir):
        with output:
            print(f"{source_dir} does not exist.")
        return

    name = parse_repo_url(url.value)["name"]
    cmd = ["ort", 
           "--debug", 
           "--stacktrace", 
           "analyze",
           "-f", "JSON", 
           "-i", f"{source_dir}/{name}", 
           "-o", f"{source_dir}/{name}-ort/analyzer", 
           "--allow-dynamic-versions"]
    if verbose:
        with output:
            print(cmd)
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if False:
        with HaloNotebook(text=f"Analyzing {' '.join(cmd)}", spinner='dots'):
            while proc.poll() is None:
                line = proc.stdout.readline().decode("utf-8")
                if False: # verbose:
                    with output:
                        print(line.strip())
    status_spinner.start(f"Analyzing {name} ...")
    while proc.poll() is None:
        line = proc.stdout.readline().decode("utf-8")
        if False: # verbose:
            with output:
                print(line.strip())
    status_spinner.succeed(f"Analyzed {name}")

    with output:
        h = f'<a href="{source_dir}/{name}-ort/analyzer/analyzer-result.json">analyser-result.json</a>'
        display(HTML2(h), display_id='myhtml')

    plot_licenses(url)

In [ ]:
def create_ort_config(path):
    """Create a (now mandatory) ORT config file.
    """
    content = textwrap.dedent("""
    excludes:
      scopes:
      - pattern: "devDependencies"
        reason: "DEV_DEPENDENCY_OF"
        comment: "Packages for development only."
    """.strip())
    open(path, "w").write(content)

In [ ]:
@output.capture()
def scan_clicked(button,
                 url=None,
                 source_dir=None,
                 verbose=False):
    """Scan a repo.
    """
    source_dir = expanduser(source_dir or ".")
    if not os.path.exists(source_dir):
        with output:
            print(f"{source_dir} does not exist.")
        return

    name = parse_repo_url(url.value)["name"]

    create_ort_config(f"{source_dir}/{name}/.ort.yml")
    
    cmd = ["ort", 
           "--debug", 
           "--stacktrace", 
           "scan",
           "-f", "JSON", 
           "-i", f"{source_dir}/{name}-ort/analyzer/analyzer-result.json", 
           "-o", f"{source_dir}/{name}-ort/scanner", 
           "--skip-excluded"
    ]
    if verbose:
        with output:
            print(cmd)
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if False:
        with HaloNotebook(text=f"Scanning: {' '.join(cmd)}", spinner='dots'):
            while proc.poll() is None:
                line = proc.stdout.readline().decode("utf-8")
                if False: # verbose:
                    with output:
                        print(line.strip())
    status_spinner.start(f"Scanning {name} ...")
    while proc.poll() is None:
        line = proc.stdout.readline().decode("utf-8")
        if False: # verbose:
            with output:
                print(line.strip())
    status_spinner.succeed(f"Scanned {name}")
    with output:
        h = f'<a href="{source_dir}/{name}-ort/scanner/scan-result.json">scanner-result.json</a>'
        display(HTML2(h), display_id='myhtml')

In [ ]:
@output.capture()
def eval_clicked(button,
                 url=None,
                 source_dir=None,
                 verbose=False):
    """Evaluate a repo.
    """
    source_dir = expanduser(source_dir or ".")
    if not os.path.exists(source_dir):
        with output:
            print(f"{source_dir} does not exist.")
        return

    name = parse_repo_url(url.value)["name"]
    cmd = ["ort", 
           "evaluate",
           "--rules-file",
           f"{source_dir}/../oss-review-toolkit/docs/examples/rules.kts",
           "--license-configuration-file",
           f"{source_dir}/../oss-review-toolkit/docs/examples/licenses.yml",
           "-f", "JSON", 
           "-i", f"{source_dir}/{name}-ort/scanner/scan-result.json", 
           "-o", f"{source_dir}/{name}-ort/evaluator", 
    ]
    if verbose:
        with output:
            print(cmd)
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if False:
        with HaloNotebook(text=f"Evaluating: {' '.join(cmd)}", spinner='dots'):
            while proc.poll() is None:
                line = proc.stdout.readline().decode("utf-8")
                if False: # verbose:
                    with output:
                        print(line.strip())
    status_spinner.start(f"Evaluating {name} ...")
    while proc.poll() is None:
        line = proc.stdout.readline().decode("utf-8")
        if False: # verbose:
            with output:
                print(line.strip())
    status_spinner.succeed(f"Evaluated {name}")
    with output:
        h = f'<a href="{source_dir}/{name}-ort/evaluator/evaluation-result.json">evaluation-result.json</a>'
        display(HTML2(h), display_id='myhtml')

In [ ]:
@output.capture()
def report_clicked(button,
                 url=None,
                 source_dir=None,
                 verbose=False):
    """Generate a report.
    """
    source_dir = expanduser(source_dir or ".")
    if not os.path.exists(source_dir):
        with output:
            print(f"{source_dir} does not exist.")
        return

    name = parse_repo_url(url.value)["name"]
    cmd = ["ort", 
           "report",
           "-f", "NoticeByPackage,StaticHtml,WebApp",
           "-i", f"{source_dir}/{name}-ort/evaluator/evaluation-result.json",
           "-o", f"{source_dir}/{name}-ort/reporter", 
    ]
    if verbose:
        with output:
            print(cmd)
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if False:
        with HaloNotebook(text=f"Reporting: {' '.join(cmd)}", spinner='dots'):
            while proc.poll() is None:
                line = proc.stdout.readline().decode("utf-8")
                if False: # verbose:
                    with output:
                        print(line.strip())
    status_spinner.start(f"Reporting {name} ...")
    while proc.poll() is None:
        line = proc.stdout.readline().decode("utf-8")
        if False: # verbose:
            with output:
                print(line.strip())
    status_spinner.succeed(f"Reported {name}")
    with output:
        h = f'<a href="{source_dir}/{name}-ort/reporter/scan-report.html">scan-report.html</a>'
        display(HTML2(h), display_id='myhtml')

In [ ]:
# defaults
repo = "https://github.com/jshttp/mime-types.git"
branch = "2.1.18"

In [ ]:
# UI
layout = Layout(width="100%")
layout1 = Layout(width="75%")
title = HTML("<strong>ORT Scan for Git Repository</strong>")
source_dd = Dropdown(description="Source", options=["", "GitHub", "GitLab", "BitBucket"])
url_tx = Text(description="Name/URL", value=repo, layout=layout1)
branch_tx = Text(description="Branch/tag/commit", value=branch, layout=layout1)
style = ButtonStyle(button_color='#48dad0')
overwrite_cb = Checkbox(description="Overwrite", value=True)
verbose_cb = Checkbox(description="Verbose", value=False)

source = source_dd.value
url = url_tx.value
branch = branch_tx.value
overwrite = overwrite_cb.value
verbose = verbose_cb.value

start_btn = Button(description="Clone", style=style)
callback = partial(clone_repo_clicked, 
                   source=source_dd, url=url_tx, branch=branch_tx, dest_dir="../tmp",
                   overwrite=overwrite, 
                   verbose=verbose)
start_btn.on_click(callback)

analyze_btn = Button(description="Analyze", style=style, disabled=not HAVE_ORT)
callback2 = partial(analyze_clicked,
                    url=url_tx, source_dir="../tmp",
                    verbose=verbose)
analyze_btn.on_click(callback2)

scan_btn = Button(description="Scan", style=style, disabled=not HAVE_ORT)
callback3 = partial(scan_clicked,
                    url=url_tx, source_dir="../tmp",
                    verbose=verbose)
scan_btn.on_click(callback3)

eval_btn = Button(description="Evaluate", style=style, disabled=not HAVE_ORT)
callback4 = partial(eval_clicked,
                    url=url_tx, source_dir="../tmp",
                    verbose=verbose)
eval_btn.on_click(callback4)

report_btn = Button(description="Report", style=style, disabled=not HAVE_ORT)
callback5 = partial(report_clicked,
                    url=url_tx, source_dir="../tmp",
                    verbose=verbose)
report_btn.on_click(callback5)

find_ort(status_spinner)
ui = VBox(children=[
    title, 
    source_dd, 
    url_tx, 
    branch_tx, 
    HBox(children=[start_btn, analyze_btn, scan_btn, eval_btn, report_btn]), 
    HBox(children=[
        # overwrite_cb,
        verbose_cb
    ]),
    output
], layout=layout)

In [ ]:
ui